Coingecko에서 데이터받기

In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from api import CoinGecko

In [2]:
gecko = CoinGecko()

In [3]:
coin_lst = gecko.get_coin_list()
coin_lst[0]

{'id': '01coin', 'symbol': 'zoc', 'name': '01coin'}

In [5]:
len(os.listdir("coingecko"))

1774

In [6]:
# 이렇게 반복문으로 받으면 된다 / 저장은 Name으로
error_list= []
dot_list = []

for coin_dict in tqdm(coin_lst[1800:5000]):
    time.sleep(5)
    try:
        df = gecko.get_coin_price_cap(coin_dict["id"])
        if "." in coin_dict["name"]:
            dot_list.append(coin_dict)
            continue
        df.to_csv("coingecko/{}.csv".format(coin_dict["name"]))
    except:
        error_list.append(coin_dict)
        continue

# dot_list랑 error_list확인해야함!!

  2%|▏         | 76/3200 [07:20<4:50:46,  5.58s/it]

In [ ]:
from api import GoogleDrive
file_lst = os.listdir("coingecko")
google = GoogleDrive("C://Jehyeon/Crypto_paper/crypto_paper/google.json/")

google.multi_upload(
    folder_name= "CoinGecko2",
    file_loc="C:/Jehyeon/Crypto_paper/crypto_paper/coingecko/",
    file_name_list=file_lst
)

In [ ]:
dot_list

In [ ]:
error_list

# 구글드라이브에서 데이터 받기

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from api import GoogleDrive

In [ ]:
google = GoogleDrive("C://Jehyeon/Crypto_paper/crypto_paper/google.json/")

In [ ]:
google_lst = google.list_files("CoinGecko")

In [ ]:
i = 0
google_dict = {}
for k,v in google_lst.items():
    i+= 1
    if i > 12060:
        google_dict[k] = v

In [ ]:
google.multi_download(
    folder_name= "CoinGecko",
    file_loc="C:/Jehyeon/Crypto_paper/crypto_paper/coingecko/",
    file_name_list=google_dict
)

In [ ]:
# 구글드라이브 데이터와 로컬의 데이터가 일치하는 지 확인
google_key = list(google_lst.keys())
not_in = []

for file in os.listdir("coingecko"):
    if file not in google_key:
        not_in.append(file)
not_in

# 로컬 데이터 전처리

In [ ]:
# timestamp를 맞춰주는 과정
not_more_five = []
execpt_list = []
for file in tqdm(os.listdir("coingecko")):
    try:
        # Unix to datetime코드
        df = pd.read_csv("coingecko/{}".format(file), index_col=0)
        if df.shape[0] < 5:
            not_more_five.append(file)
            continue
        elif len(str(df.Time[0])) == 13:
            df.Time= pd.to_datetime(df.Time, unit="ms")
            df.Time.apply(lambda x: x.strftime("%Y-%m-%d"))
            df.to_csv("coingecko_pp/pp_{}".format(file))
        else:
            df.to_csv("coingecko_pp/{}".format(file))
    except:
        execpt_list.append(file)
        continue

In [ ]:
# 병국이가 원래 datetime으로 인덱스를 저장한 것만 불러와서 pp3에 저장해야한다
exception_list = []
ar = []
for file in tqdm(os.listdir("coingecko")):
    try:
        # Unix to datetime코드
        df = pd.read_csv("coingecko/{}".format(file), index_col=0)
        if df.shape[0] < 5:
            ar.append(file)
            continue
        elif str(df.index[0]).startswith("20"):
            df.to_csv("coingecko_pp3/{}".format(file))
    except:
        exception_list.append(file)

In [ ]:
exception_list

In [ ]:
len(ar)

In [ ]:
print(len(os.listdir("coingecko")))
print(len(os.listdir("coingecko_pp2")))
print(len(os.listdir("coingecko_pp3")))
# 이제 coingecko pp2와 coingecko pp3을 합치면된다

In [ ]:
for file in tqdm(os.listdir("coingecko_pp2")):
    df = pd.read_csv("coingecko_pp2/{}".format(file))
    if str(file).startswith("pp_"):
        file = str(file)[3:]
    df.to_csv("coingecko_pp3/{}".format(file))

In [ ]:
len(os.listdir("coingecko_pp3"))

# 데이터 전처리

In [ ]:
import os 
import pandas as pd
from tqdm import tqdm

In [ ]:
# 느린코드
return_df = pd.DataFrame()

for file in tqdm(os.listdir("coingecko_pp3")[:6000]):
    try:
        df = pd.read_csv("coingecko_pp3/{}".format(file), index_col=0)[["Price","Time"]]
        df.index = pd.to_datetime(df.Time)
        df = df.drop("Time",axis=1)
        col = list(return_df.columns)

        return_df = pd.merge(return_df,df,
                             left_index=True, right_index=True,
                             how="outer")
        col.append(str(file[:-4]))
        return_df.columns = col
    except:
        continue
return_df

In [ ]:
# 빠른코드
coin_dict = {}

for file in tqdm(os.listdir("coingecko_pp3")):
    try: # 내가 저장
        df = pd.read_csv("coingecko_pp3/{}".format(file), index_col=0)[["Time","Price","Marketcap"]]
        df.index = pd.to_datetime(df.Time)
        df.index = df.index.strftime("%Y-%m-%d")
        df.drop("Time", axis=1, inplace=True)
        df.columns = ["prices","market_caps"]
        coin_dict[file] = df
    except: # 병국이 저장
        df = pd.read_csv("coingecko_pp3/{}".format(file), index_col=0)[["prices","market_caps"]]
        df.index = pd.to_datetime(df.index)
        df.index = df.index.strftime("%Y-%m-%d")
        coin_dict[file] = df

In [ ]:
coin_price_dict = {}
for key,df in coin_dict.items():
    coin_price_dict[key] = df[["prices"]]

In [ ]:
# find the longest timeframe index
longest_index = max([df.index for df in coin_price_dict.values()], key=len)

# create a close dataframe with the longest timeframe index and columns "key1", "key2", "key3"
close_df = pd.DataFrame(index=longest_index, columns=coin_price_dict.keys())

for key, df in tqdm(coin_price_dict.items()):
    # use groupby and last to select the last value for each index
    close_df[key] = df.groupby(df.index).last().reindex(longest_index)

In [ ]:
print(os.listdir("coingecko_pp3"))

In [ ]:
#close_df["ethereum.csv"].plot()

# 회귀분석

In [ ]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
# 데이터 불러오고 합치기
#coin_dict = {}
#
#file_list = os.listdir("data")
#
#for f in file_list:
#    print(f)
#    df = pd.read_csv("Data/" + f, index_col=0,
#                     parse_dates=True)["close"]
#    coin_dict[str(f)[:-4]] = df
#
#coin_df = pd.DataFrame(coin_dict)

In [ ]:
# 합친 데이터 불러오기
coin_df = pd.read_csv("all_coin_closeprice.csv", index_col=0,
                      parse_dates=True)

# 0은 NaN으로 변경
coin_df_with_nan = coin_df.replace(0,np.nan)

In [ ]:
# 주간 리턴 계산
save_dict = {}
week_start  = pd.date_range("2010-01-01","2022-12-31", freq="W")

for i in range(len(week_start)-1):
    w_start = week_start[i]
    w_end = week_start[i+1] - pd.Timedelta(days=1)
    tmp_df = coin_df_with_nan.loc[w_start:w_end] # 일주일치 데이터 추출
    w_rtn = tmp_df.apply(lambda x: (x[-1] - x[0]) / x[0] + 1, axis=0) # 일주일간 리턴 계산
    save_dict[w_start] = w_rtn

weekly_return = pd.DataFrame(save_dict).T

In [ ]:
# row별로 rank를 때려줌(수익률이 높을수록 0에 가까움) (NaN은 처리함)
weekly_return_rank = weekly_return.rank(axis=1, pct=True, 
                                        ascending=False, na_option="keep") 
weekly_return_rank

In [ ]:
# rank를 기준으로 screening한 DataFrame을 만든다
high = weekly_return_rank.apply(lambda x: np.where(x < 1/3, 1, np.nan))
middle = weekly_return_rank.apply(lambda x: np.where(((x < 2/3) & (x >= 1/3)), 1, np.nan))
low = weekly_return_rank.apply(lambda x: np.where(x >= 2/3, 1, np.nan))

In [ ]:
# 분위수별 주간 리턴 구하기(평균처줌)
high_rtn = (weekly_return * high).mean(skipna=True, axis=1)
middle_rtn = (weekly_return * middle).mean(skipna=True, axis=1)
low_rtn = (weekly_return * low).mean(skipna=True, axis=1)

In [ ]:
# Sample기간 동안의 수익률 구하기 (기간을 길게 잡으니 inf가 뜸)
a = high_rtn.loc["2020":"2022"].cumprod()
b = middle_rtn.loc["2020":"2022"].cumprod()
c = low_rtn.loc["2020":"2022"].cumprod()

In [ ]:
def do_regression(df, t):
    '''t만큼 시차를 가진 regression'''
    tmp_df = df.copy()
    y = tmp_df.shift(-t)
    concat = pd.concat([tmp_df, y], keys=["return","return_"+str(t)], axis=1) \
               .dropna()
    model = sm.OLS(concat["return_"+str(t)], concat["return"])
    result = model.fit(cov_type='HAC',cov_kwds={'maxlags':t}) #t-1인가?
    return result

In [ ]:
low_1 = do_regression(a, 1)
low_2 = do_regression(a, 2)
low_4 = do_regression(a, 4)

middle_1 = do_regression(b, 1)
middle_2 = do_regression(b, 2)
middle_4 = do_regression(b, 4)

high_1 = do_regression(c, 1)
high_2 = do_regression(c, 2)
high_4 = do_regression(c, 4)
#rt1_1.params, rt1_1.pvalues

In [ ]:
pd.DataFrame({"R_t_t+1": [low_1.params[0], middle_1.params[0], high_1.params[0]],
              "R_t_t+2": [low_2.params[0], middle_2.params[0], high_2.params[0]],
              "R_t_t+4": [low_4.params[0], middle_4.params[0], high_4.params[0]]},
              index=["Low","Middle","Hogh"])